# 台灣證券交易所
- NCCU PyDay 2022: https://www.facebook.com/groups/pythontw/posts/10162007038723438/
- Author: TENG-LIN YU
- Email: tlyu0419@gmail.com
- Github: https://github.com/TLYu0419/Stock-Prediction-Using-Facebook-Sentiment-with-Python

In [13]:
import time
import requests
import pandas as pd
import datetime
import re
import numpy as np

In [14]:
df = []
rs = requests.Session()
headers={'Referer': 'https://www.twse.com.tw/zh/',
         'headers':'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36 Edg/100.0.1185.39'}
    
for date in pd.date_range('2021-12-01', '2022-04-01', freq='MS'):
    url = 'https://www.twse.com.tw/en/indicesReport/MI_5MINS_HIST?response=json&date={}'.format(date.strftime('%Y%m%d'))
    print(url)
    resp = rs.get(url, headers=headers)
    ndf = pd.DataFrame(resp.json()['data'],           
                       columns=resp.json()['fields'])
    ndf['title'] = resp.json()['title']
    time.sleep(3)
    df.append(ndf)
df = pd.concat(df, ignore_index=True)
df

https://www.twse.com.tw/en/indicesReport/MI_5MINS_HIST?response=json&date=20211201
https://www.twse.com.tw/en/indicesReport/MI_5MINS_HIST?response=json&date=20220101
https://www.twse.com.tw/en/indicesReport/MI_5MINS_HIST?response=json&date=20220201
https://www.twse.com.tw/en/indicesReport/MI_5MINS_HIST?response=json&date=20220301
https://www.twse.com.tw/en/indicesReport/MI_5MINS_HIST?response=json&date=20220401


,Date,Opening Index,Highest Index,Lowest Index,Closing Index,title
0,2021/12/01,"17,428.61","17,626.21","17,374.59","17,585.99",2021/12 TAIEX Total Index Historical Data
1,2021/12/02,"17,575.91","17,741.55","17,559.44","17,724.88",2021/12 TAIEX Total Index Historical Data
2,2021/12/03,"17,720.93","17,781.14","17,670.32","17,697.14",2021/12 TAIEX Total Index Historical Data
3,2021/12/06,"17,650.67","17,763.42","17,585.95","17,688.21",2021/12 TAIEX Total Index Historical Data
4,2021/12/07,"17,734.19","17,796.92","17,642.32","17,796.92",2021/12 TAIEX Total Index Historical Data
...,...,...,...,...,...,...
83,2022/04/12,"17,005.84","17,100.85","16,905.13","16,990.91",2022/04 TAIEX Total Index Historical Data
84,2022/04/13,"17,080.40","17,333.21","17,080.40","17,301.65",2022/04 TAIEX Total Index Historical Data
85,2022/04/14,"17,353.25","17,374.14","17,245.65","17,245.65",2022/04 TAIEX Total Index Historical Data
86,2022/04/15,"17,139.34","17,141.33","17,004.18","17,004.18",2022/04 TAIEX Total Index Historical Data


In [15]:
df['Date'] = df['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y/%m/%d'))
df['Opening Index'] = df['Opening Index'].apply(lambda x: float(re.sub(',', '', x)))
df['Highest Index'] = df['Highest Index'].apply(lambda x: float(re.sub(',', '', x)))
df['Lowest Index'] = df['Lowest Index'].apply(lambda x: float(re.sub(',', '', x)))
df['Closing Index'] = df['Closing Index'].apply(lambda x: float(re.sub(',', '', x)))
df.columns = [col.upper() for col in df.columns]
df.columns = [re.sub(' ', '_', col) for col in df.columns]
df

,DATE,OPENING_INDEX,HIGHEST_INDEX,LOWEST_INDEX,CLOSING_INDEX,TITLE
0,2021-12-01,17428.61,17626.21,17374.59,17585.99,2021/12 TAIEX Total Index Historical Data
1,2021-12-02,17575.91,17741.55,17559.44,17724.88,2021/12 TAIEX Total Index Historical Data
2,2021-12-03,17720.93,17781.14,17670.32,17697.14,2021/12 TAIEX Total Index Historical Data
3,2021-12-06,17650.67,17763.42,17585.95,17688.21,2021/12 TAIEX Total Index Historical Data
4,2021-12-07,17734.19,17796.92,17642.32,17796.92,2021/12 TAIEX Total Index Historical Data
...,...,...,...,...,...,...
83,2022-04-12,17005.84,17100.85,16905.13,16990.91,2022/04 TAIEX Total Index Historical Data
84,2022-04-13,17080.40,17333.21,17080.40,17301.65,2022/04 TAIEX Total Index Historical Data
85,2022-04-14,17353.25,17374.14,17245.65,17245.65,2022/04 TAIEX Total Index Historical Data
86,2022-04-15,17139.34,17141.33,17004.18,17004.18,2022/04 TAIEX Total Index Historical Data


In [16]:
# EDA 繪製股價變化圖

## 活動
- 如何計算連續的休假天數?

In [17]:
stock_price = pd.DataFrame(pd.date_range('2019-12-01', '2022-04-18', freq='D'), columns=['DATE'])
stock_price = pd.merge(stock_price, df[['DATE', 'CLOSING_INDEX']], how='left', on='DATE')
stock_price = stock_price.loc[stock_price['CLOSING_INDEX'].notnull()]
stock_price['LAST_DATE'] = stock_price['DATE'].shift(1)
stock_price['DAYS'] = stock_price['DATE'] - stock_price['LAST_DATE']
stock_price['DAYS'] = stock_price['DAYS'].apply(lambda x: x.days)
stock_price['DAYS'] = stock_price['DAYS']-1
stock_price.head(30)

,DATE,CLOSING_INDEX,LAST_DATE,DAYS
731,2021-12-01,17585.99,NaT,NaN
732,2021-12-02,17724.88,2021-12-01,0.0
733,2021-12-03,17697.14,2021-12-02,0.0
736,2021-12-06,17688.21,2021-12-03,2.0
737,2021-12-07,17796.92,2021-12-06,0.0
738,2021-12-08,17832.42,2021-12-07,0.0
739,2021-12-09,17914.12,2021-12-08,0.0
740,2021-12-10,17826.26,2021-12-09,0.0
743,2021-12-13,17767.60,2021-12-10,2.0
744,2021-12-14,17599.37,2021-12-13,0.0


In [18]:
stock_price['DAYS'].value_counts()

0.0     68
2.0     15
3.0      2
11.0     1
4.0      1
Name: DAYS, dtype: int64

# convex function

In [19]:
closings = [2, np.nan, 6, 8, 10, np.nan, np.nan, np.nan, 18, 20]
for i in range(len(closings)):
    if str(closings[i]) != 'nan':
        pass
    if str(closings[i]) == 'nan':
        last_ava = closings[i-1]
        j = 1
        while True:
            if str(closings[i+j]) != 'nan':
                next_ava = closings[i+j]
                break
            j = j+1
        closings[i] = (last_ava+next_ava)/2
closings

[2, 4.0, 6, 8, 10, 14.0, 16.0, 17.0, 18, 20]

In [20]:
## 如何處理遺漏值 - 週末、假日...
stock_price = pd.DataFrame(pd.date_range('2021-06-01', '2022-04-18', freq='D'), columns=['DATE'])
stock_price = pd.merge(stock_price, df[['DATE', 'CLOSING_INDEX']], how='left', on='DATE')
stock_price.head(50)

,DATE,CLOSING_INDEX
0,2021-06-01,NaN
1,2021-06-02,NaN
2,2021-06-03,NaN
3,2021-06-04,NaN
4,2021-06-05,NaN
5,2021-06-06,NaN
6,2021-06-07,NaN
7,2021-06-08,NaN
8,2021-06-09,NaN
9,2021-06-10,NaN


In [21]:
stock_price = stock_price.loc[stock_price['DATE']!='2019-12-01']
for i in range(stock_price.shape[0]):
    if str(stock_price['CLOSING_INDEX'].values[i]) != 'nan':
        pass
    if str(stock_price['CLOSING_INDEX'].values[i]) == 'nan':
        last_ava = stock_price['CLOSING_INDEX'].values[i-1]
        
        j = 1
        while True:
            if str(stock_price['CLOSING_INDEX'].values[i+j]) != 'nan':
                next_ava = stock_price['CLOSING_INDEX'].values[i+j]
                break
            j = j+1
        stock_price.iloc[i, 1] = (last_ava+next_ava)/2
stock_price

,DATE,CLOSING_INDEX
0,2021-06-01,17242.4300
1,2021-06-02,17414.2100
2,2021-06-03,17500.1000
3,2021-06-04,17543.0450
4,2021-06-05,17564.5175
...,...,...
317,2022-04-14,17245.6500
318,2022-04-15,17004.1800
319,2022-04-16,16951.5250
320,2022-04-17,16925.1975


In [22]:
stock_price.head(20)

,DATE,CLOSING_INDEX
0,2021-06-01,17242.430000
1,2021-06-02,17414.210000
2,2021-06-03,17500.100000
3,2021-06-04,17543.045000
4,2021-06-05,17564.517500
5,2021-06-06,17575.253750
6,2021-06-07,17580.621875
7,2021-06-08,17583.305938
8,2021-06-09,17584.647969
9,2021-06-10,17585.318984


In [31]:
# 驗證
5	2019-12-06	11609.64
6	2019-12-07	NaN
7	2019-12-08	NaN
8	2019-12-09	11660.77

SyntaxError: invalid syntax (2307642925.py, line 2)

In [32]:
(11609.64 + 11660.77)/2

11635.205

In [33]:
(11635.205 + 11660.77)/2

11647.9875

In [23]:
stock_price.to_pickle('./stock_price_by_date.pickle')